# Build an Agent from Scratch

In [1]:
import re
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

## The Reasoning + Acting (React) pattern

Based on https://til.simonwillison.net/llms/python-react-pattern

In [2]:
client = OpenAI()

In [3]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=self.messages,
        )
        # print(completion.usage)
        return completion.choices[0].message.content

In [4]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [5]:
# Tools to provide to our agent (actions)
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

### Example 1

In [6]:
# Initialize the agent
agent = Agent(prompt)

In [7]:
result = agent("How much does a scottish terrier weigh?")
print(result)

Thought: I should look up the average weight of a Scottish Terrier using the average_dog_weight action.
Action: average_dog_weight: Scottish Terrier
PAUSE


In [8]:
result = average_dog_weight("Scottish Terrier")
result

'Scottish Terriers average 20 lbs'

In [9]:
next_prompt = "Observation: " + result
agent(next_prompt)

'Answer: A Scottish Terrier weighs 20 lbs'

In [10]:
agent.messages[1:]

[{'role': 'user', 'content': 'How much does a scottish terrier weigh?'},
 {'role': 'assistant',
  'content': 'Thought: I should look up the average weight of a Scottish Terrier using the average_dog_weight action.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'},
 {'role': 'user', 'content': 'Observation: Scottish Terriers average 20 lbs'},
 {'role': 'assistant', 'content': 'Answer: A Scottish Terrier weighs 20 lbs'}]

### Example 2

In [11]:
# Init + question
agent = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
print(agent(question))

Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight action, and then calculate their combined weight.

Action: average_dog_weight: Border Collie
PAUSE


In [12]:
# Action 1
observation = average_dog_weight("Border Collie")
next_prompt = f"Observation: {observation}"
print(agent(next_prompt))

Action: average_dog_weight: Scottish Terrier
PAUSE


In [13]:
# Action 2
observation = average_dog_weight("Scottish Terrier")
next_prompt = f"Observation: {observation}"
print(agent(next_prompt))

Action: calculate: 37 + 20
PAUSE


In [14]:
# Action 3 + answer
observation = calculate("37 + 20")
next_prompt = f"Observation: {observation}"
print(agent(next_prompt))

Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs


### Automate the loop

In [15]:
# A regex to get the exact action name and params
action_re = re.compile(r"^Action: (\w+): (.*)$")

# Querying loop
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i+=1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(line)
            for line in result.splitlines()
            if action_re.match(line)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {action}: {action_input}")
            print(" -- running {action}: {action_input}")
            observation = known_actions[action](action_input)
            next_prompt = f"Observation: {observation}"
        else:
            return

In [16]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight action, and then calculate their combined weight.

Action: average_dog_weight: Border Collie
PAUSE
 -- running {action}: {action_input}
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running {action}: {action_input}
Action: calculate: 37 + 20
PAUSE
 -- running {action}: {action_input}
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs
